In [ ]:
"""
Fibonacci Retracement Trading Strategy Implementation

This script implements a trading strategy based on Fibonacci retracement levels, 
which are horizontal lines indicating potential support/resistance levels.
The strategy fetches stock price data, calculates Fibonacci levels from swing points,
generates trading signals, and visualizes the results.

Dependencies:
- pandas
- numpy
- matplotlib
- yfinance
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


def get_stock_data(ticker, start_date, end_date):
    """
    Fetch and preprocess stock price data from Yahoo Finance.
    
    Args:
        ticker (str): Stock ticker symbol (e.g., 'AAPL')
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    
    Returns:
        pd.DataFrame: DataFrame containing Date, Close, High, Low, Open prices
    """
    data = yf.download(ticker, start= start_date, end= end_date).reset_index()
    data.columns = data.columns.get_level_values(0)  # Flatten multi-index
    return data[['Date', 'Close', 'High', 'Low', 'Open']]


def calculate_fib_levels(swing_high, swing_low):
    """
    Calculate Fibonacci retracement levels based on swing high and low prices.
    
    Args:
        swing_high (float): Highest price in the lookback period
        swing_low (float): Lowest price in the lookback period
    
    Returns:
        dict: Dictionary of Fibonacci levels with percentages as keys and prices as values
    """
    price_range = swing_high - swing_low
    return {
        '0%': swing_high,
        '23.6%': swing_high - price_range * 0.236,
        '38.2%': swing_high - price_range * 0.382,
        '50%': swing_high - price_range * 0.5,
        '61.8%': swing_high - price_range * 0.618,
        '100%': swing_low
    }


def fib_trading_strategy(data, lookback_period= 30):
    """
    Implement Fibonacci trading strategy by generating signals based on current price
    relative to Fibonacci retracement levels.
    
    Args:
        data (pd.DataFrame): DataFrame containing stock price data
        lookback_period (int): Number of days to look back for swing high/low (default: 30)
    
    Returns:
        tuple: (fib_levels, current_price, signal)
            fib_levels: Dictionary of Fibonacci levels
            current_price: Latest closing price
            signal: Generated trading signal
    """
    # Identify recent swing high and low based on 30 day period
    swing_high = data['High'].rolling(lookback_period).max().iloc[-1]
    swing_low = data['Low'].rolling(lookback_period).min().iloc[-1]

    # Calculate Fibonacci retracement levels
    fib_levels = calculate_fib_levels(swing_high, swing_low)

    # Get the current price
    current_price = data['Close'].iloc[-1]

    # Generate signals based on current price relative to Fibonacci levels
    signal = "Hold"
    if current_price <= fib_levels['38.2%'] and current_price > fib_levels['50%']:
        signal = "Buy (Weak Support)"
    elif current_price <= fib_levels['50%'] and current_price > fib_levels['61.8%']:
        signal = "Buy (Strong Support)"
    elif current_price <= fib_levels['61.8%']:
        signal = "Buy (Very Strong Support)"
    elif current_price >= fib_levels['0%']:
        signal = "Consider Taking Profits"

    return fib_levels, current_price, signal


def visualize_strategy(data, fib_levels, current_price, signal, lookback_period= 30):
    """
    Visualize the Fibonacci trading strategy with price data, Fibonacci levels,
    swing points, and trading signal.
    
    Args:
        data (pd.DataFrame): DataFrame containing stock price data
        fib_levels (dict): Dictionary of Fibonacci levels
        current_price (float): Latest closing price
        signal (str): Generated trading signal
        lookback_period (int): Number of days used for swing high/low calculation
    """
    plt.figure(figsize=(14, 8))
    
    # Plot closing price
    plt.plot(data['Date'], data['Close'], label= 'Closing Price', color= 'blue', alpha= 0.7)
    
    # Plot Fibonacci levels as horizontal lines
    colors = ['red', 'orange', 'green', 'blue', 'purple', 'black']
    for i, (level, price) in enumerate(fib_levels.items()):
        plt.axhline(y= price, color= colors[i], linestyle= '--', 
                   label= f'Fib {level} (${price:.2f})', alpha= 0.7)
    
    # Highlight current price
    plt.axhline(y= current_price, color= 'black', linestyle= '-', 
               label= f'Current Price (${current_price:.2f})', alpha= 1)
    
    # Mark the swing high and low points
    swing_high = data['High'].rolling(lookback_period).max().iloc[-1]
    swing_low = data['Low'].rolling(lookback_period).min().iloc[-1]
    swing_high_date = data.loc[data['High'] == swing_high, 'Date'].iloc[-1]
    swing_low_date = data.loc[data['Low'] == swing_low, 'Date'].iloc[-1]
    
    plt.scatter(swing_high_date, swing_high, color='red', s= 100, 
               label= f'Swing High (${swing_high:.2f})', zorder= 5)
    plt.scatter(swing_low_date, swing_low, color='green', s= 100, 
               label= f'Swing Low (${swing_low:.2f})', zorder= 5)
    
    # Add signal annotation
    plt.annotate(f'SIGNAL: {signal}', 
                xy= (data['Date'].iloc[-1], current_price),
                xytext= (10, 10), textcoords= 'offset points',
                bbox=dict(boxstyle= 'round', facecolor= 'yellow', alpha= 0.8),
                fontsize= 12, fontweight= 'bold')
    
    # Formatting
    plt.title('Fibonacci Trading Strategy', fontsize= 16)
    plt.xlabel('Date', fontsize= 12)
    plt.ylabel('Price ($)', fontsize= 12)
    plt.grid(True, alpha= 0.3)
    plt.legend(loc= 'lower right')
    plt.tight_layout()
    
    # Show plot
    plt.show()


if __name__ == "__main__":
    """
    Main execution block for testing the Fibonacci trading strategy.
    """
    # Get data
    stock_data = get_stock_data('AAPL', '2023-01-01', '2024-01-01')

    # Run strategy
    fib_levels, current_price, signal = fib_trading_strategy(stock_data)

    # Display results
    print("\nFibonacci Trading Strategy for AAPL")
    print("----------------------------------")
    print(f"Current Price: ${current_price}")
    print("\nFibonacci Levels:")
    for level, price in fib_levels.items():
        print(f"{level}: ${price}")
    print(f"\nSignal: {signal}")

    # Visualize results
    visualize_strategy(stock_data, fib_levels, current_price, signal)